# Import aller nötigen Module

In [1]:
import pandas as pd
import numpy as np
import joblib
import plotly.express as px
from scipy.signal import butter, filtfilt
from scipy import stats
from collections import Counter
import sys
import os

# Initialisierung des Dobot-Roboterarm

In [ ]:
sys.path.insert(0, os.path.abspath('.'))

from lib.dobot import Dobot
bot = Dobot('COM3')

# Hilfsfunktion
Hilfsfunktion zum Plotten

In [2]:
# Vordefinierte Hilfsfunktion zum plotten
def plot_filter(dataframe, data_column_name, filtered_column_name, title):
    fig = px.line(dataframe, x='Time (s)', y=[data_column_name, filtered_column_name], title=title)
    fig.update_xaxes(title_text='Time')
    fig.update_yaxes(title_text=filtered_column_name)
    fig.show() 

# Datenimport

Hier laden wir unsere .pyphox Dateien und fangen ab, falls diese nicht vorhanden sein solte. Zudem setzen wir schoonmal die Spaltennamen fest um diese im fortlaufenen Programm nutzen zu können

In [ ]:
# Error-Handling
try:
    new_data = pd.read_csv("Raw Data.csv")
except FileNotFoundError:
    print(f"Error: File not found")
    exit()

# Vorschau der Daten
print("Dataset:")
print(new_data)

Dataset:
     Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0    0.096974                          0.123                          0.101   
1    0.101955                          0.127                          0.105   
2    0.106936                          0.131                          0.109   
3    0.111917                          0.145                          0.123   
4    0.116898                          0.162                          0.139   
..        ...                            ...                            ...   
981  4.983422                          0.000                         -0.013   
982  4.988403                         -0.001                         -0.013   
983  4.993384                          0.003                          0.001   
984  4.998365                          0.003                         -0.015   
985  5.003346                          0.010                         -0.013   

     Linear Acceleration z (m/s^2)  Absolu

# Frequenzfilter

Hier wird das Rauschen reduziert. Dafür nutzen wir Butterworth aus der Vorlesung. Damit werden hoch- bzw. niederfrequente Signale gefiltert. 
Zuerst folgen die niederfrequenten Signale, dann die hochfrequenten:

In [11]:
filtered_new_data = new_data.copy()

filter_order = 4
low_cutoff = 0.5
high_cutoff = 5

# Hier nutzen wir den butterwort filter um die Daten zu glätten
b, a = butter(filter_order, [low_cutoff, high_cutoff], btype='band', analog=False, fs=100)

# Wende den Filter auf die Beschleunigungssignale an
filtered_new_data['filtered_X_band'] = filtfilt(b, a, new_data['Linear Acceleration x (m/s^2)'])
filtered_new_data['filtered_Y_band'] = filtfilt(b, a, new_data['Linear Acceleration y (m/s^2)'])
filtered_new_data['filtered_Z_band'] = filtfilt(b, a, new_data['Linear Acceleration z (m/s^2)'])

plot_filter(filtered_new_data, 'Linear Acceleration x (m/s^2)', 'filtered_X_band', 'Acceleration X with Band-Pass Filtering')
plot_filter(filtered_new_data, 'Linear Acceleration y (m/s^2)', 'filtered_Y_band', 'Acceleration Y with Band-Pass Filtering')
plot_filter(filtered_new_data, 'Linear Acceleration z (m/s^2)', 'filtered_Z_band', 'Acceleration Z with Band-Pass Filtering')

# Ausreißer Bereinigung
Hier wollen wir nun die möglichen Ausreißer bereinigen: 

In [12]:
# Berechnung der Z-Scores für jede Spalte (X, Y, Z)
filtered_new_data['Z_Score_X_band'] = stats.zscore(filtered_new_data['filtered_X_band'])
filtered_new_data['Z_Score_Y_band'] = stats.zscore(filtered_new_data['filtered_Y_band'])
filtered_new_data['Z_Score_Z_band'] = stats.zscore(filtered_new_data['filtered_Z_band'])

# Festlegen eines Schwellenwerts für Z-Scores, ab dem ein Punkt als Ausreißer betrachtet wird
z_score_threshold = 4

# Filtern der Ausreißer aus dem filterd_new_data (nur wenn alle drei Achsen unterhalb des Threshold liegen sind Daten ok)
new_data = new_data[(abs(stats.zscore(new_data['Linear Acceleration x (m/s^2)'])) <= z_score_threshold) &
                    (abs(stats.zscore(new_data['Linear Acceleration y (m/s^2)'])) <= z_score_threshold) &
                    (abs(stats.zscore(new_data['Linear Acceleration z (m/s^2)'])) <= z_score_threshold)]

# Jetzt enthält 'filtered_new_data' nur noch die Datenpunkte, die nicht als Ausreißer betrachtet werden.
print(filtered_new_data.shape)

#Plotting
plot_filter(filtered_new_data, 'Linear Acceleration x (m/s^2)', 'Z_Score_X_band', 'Acceleration X with Z-Score Filtering')
plot_filter(filtered_new_data, 'Linear Acceleration y (m/s^2)', 'Z_Score_Y_band', 'Acceleration Y with Z-Score Filtering')
plot_filter(filtered_new_data, 'Linear Acceleration z (m/s^2)', 'Z_Score_Z_band', 'Acceleration Z with Z-Score Filtering')

(986, 11)


# Windowing
Hier nutzen wir Windowing um die Daten zu zerlegen. Das brauchen wir später für die Features-Berechnung

In [13]:
# Da fs = 100 Hz setzen wir das Fenster auf 0.5 Sekunden um auch kurze Bewegungsereignisse abzubilden. 
# Wir könnten window_size aber auch auf 200 setzen.
window_size = 150
overlap = 25

#Für jedes Fenster in den Daten neuen Dataframe erstellen, der in Liste abgelegt wird 
def create_windows(df):
    windows = []
    for i in range(0, len(df), window_size - overlap):
        window = df.iloc[i:i+window_size]
        if len(window) == window_size:
            windows.append(window)
    return windows

windows = create_windows(filtered_new_data)

#Ersten Dataframe in der Liste ausgeben lassen 
print("Dataframe: ")
print(windows[0].head())
#Anzahl Datenpunkte des ersten Dataframes ausgeben lassen 
print("Datenpunkte: ")
print(len(windows[0]))
#Spalten des ersten Dataframes ausgeben lassen 
print("Spalten: ")
print(windows[0].columns)

#Ausgeben lassen wie viele Dataframes also Fenster entstanden sind 
print("Dataframes: ")
print (len(windows))

Dataframe: 
   Time (s)  Linear Acceleration x (m/s^2)  Linear Acceleration y (m/s^2)  \
0  0.096974                          0.123                          0.101   
1  0.101955                          0.127                          0.105   
2  0.106936                          0.131                          0.109   
3  0.111917                          0.145                          0.123   
4  0.116898                          0.162                          0.139   

   Linear Acceleration z (m/s^2)  Absolute acceleration (m/s^2)  \
0                          0.021                       0.160533   
1                          0.023                       0.166382   
2                          0.026                       0.172389   
3                          0.032                       0.192816   
4                          0.037                       0.216643   

   filtered_X_band  filtered_Y_band  filtered_Z_band  Z_Score_X_band  \
0        -1.204950        -0.702109        -0.1339

# Features erstellen
Hier berechnen wir die Merkmale, die wesentliche Eigneschaften der Daten kompakt beschreiben. Diese dienen als Eingabe für den späteren Entscheidungsbaum. 

In [14]:
# Funktion zur Feature-Extraktion aus Fenster-Daten
def extract_features(windows):
    # Leere Liste für die Feature-Dictionaries
    feature_list = []
    # Schleife über alle Fenster
    for window in windows:
        # Berechnung der Features für X, Y, Z Achsen
        features = {
            'BeschleunigungX_Mittelwert': window['filtered_X_band'].mean(),
            'BeschleunigungY_Mittelwert': window['filtered_Y_band'].mean(),
            'BeschleunigungZ_Mittelwert': window['filtered_Z_band'].mean(),
            'BeschleunigungX_Std': window['filtered_X_band'].std(),
            'BeschleunigungY_Std': window['filtered_Y_band'].std(),
            'BeschleunigungZ_Std': window['filtered_Z_band'].std(),
            'BeschleunigungX_Max': window['filtered_X_band'].max(),
            'BeschleunigungY_Max': window['filtered_Y_band'].max(),
            'BeschleunigungZ_Max': window['filtered_Z_band'].max(),
            'BeschleunigungX_Min': window['filtered_X_band'].min(),
            'BeschleunigungY_Min': window['filtered_Y_band'].min(),
            'BeschleunigungZ_Min': window['filtered_Z_band'].min(),
        }
        # Features des Fensters der Liste hinzufügen
        feature_list.append(features)
    # Alle Features in einen DataFrame umwandeln und zurückgeben
    return pd.DataFrame(feature_list)

features_df = extract_features(windows)

# Entscheidungsbaum laden
Hier wird der Entscheidungsbaum geladen

In [15]:
dt = joblib.load("decision_tree_model.pkl")

# Vorhersagen
Hier wird Vorhergesagt, welche Bewegung durchgeführt wurde. 

In [16]:
# Vorhersage für alle Fenster
y_pred = dt.predict(features_df)
print(y_pred)

# Gesamtbewegung bestimmen
overall_pred = Counter(y_pred).most_common(1)[0][0]
print("Bewegung erkannt:")
print(overall_pred)

['Hammer' 'Wischen' 'Wischen' 'Wischen' 'Wischen' 'Hammer' 'Hammer']
Bewegung erkannt:
Wischen


# Dobot-Funktion für die Bewegung (Wischen)
Diese Funktion wird ausgeführt, wenn eine Wisch Bewegung erkannt wurde.

In [ ]:
# Wenn eine Wischbewegung erkannt wurde dann wird ein Kasten, in dem ein Haken ist gezeichent.
def wischen(bot):
    print("Aktion: Wischen")
    # Initialisierte Werte für den Kasten + Haken
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    # --- Kasten zeichnen
    # fährt zur ersten Position um den Kasten zu zeichnen
    print('Kasten-Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    # fährt den Arm runter um mit den Zeichnen zu beginnen
    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    # fährt zur zweiten Position
    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    # fährt zur dritten Position
    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    # fährt zur vierten Position
    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    # fährt zurück zur ersten Position
    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    # --- Haken zeichnen
    # fährt den Arm hoch
    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    # fährt zur ersten Position um den Haken zu zeichen
    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    # fährt den Arm runter um mit den Zeichnen zu beginnen
    print('Haken_Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    # fährt zur ersten Position
    print('Haken_Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    # fährt zur zweiten Position
    print('Haken_Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    # --- Ende
    # fährt den Arm hoch
    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

# Dobot-Funktion für die Bewegung (Hammer)
Diese Funktion wird ausgeführt, wenn eine Hammer Bewegung erkannt wurde.

In [ ]:
# Wenn eine Hammerbewegung erkannt wurde dann wird ein Kasten, in dem ein Kreuz ist gezeichent.
def hammer(bot):
    print("Aktion: Hammer")
    # Initialisierte Werte für den Kasten + Kreuz
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    # --- Kasten zeichnen
    # fährt zur Startposition um den Kasten zu zeichnen
    print('Kasten-Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    # fährt den Arm runter um mit den Zeichnen zu beginnen
    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    # fährt zur zweiten Position
    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    # fährt zur dritten Position
    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    # fährt zur vierten Position
    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    # fährt zur ersten Position zurück
    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    # --- Kreuz zeichnen
    # fährt zur ersten Position vom Kreuz
    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    # fährt hoch
    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    # fährt zur zweiten Position vom Kreuz
    print('Kreuz-Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    # fährt den Arm runter um mit dem Zeichen fortzufahren
    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    # fährt zur dritten Position vom Kreuz
    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    # --- Ende
    # fährt den Arm hoch
    print('Hochfahren')
    bot.move_to(x_min, y_min, z_max, 0)


# Dobot Aktionen anhand der Vorhersage ausführen

Hier verknüpfen wir die prediction mit dem Bot

In [ ]:
# Rekalibriert sich
print('Homing-Modus')
bot.home()

# Jenachdem was für eine Bewegung durch den Entscheidungsbaum erkannt wurde, wird die jeweilige Zeichnung gemacht.
if overall_pred == "Hammer":    # Kreuz
    hammer(bot)
else:                           # Haken
    wischen(bot)

# Wenn der Code oben nicht funktioniert mit dem Roboter

In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('.'))

from lib.dobot import Dobot

bot = Dobot('COM3')

def wischen(bot):
    print("Aktion: Wischen")
    x_min = 220
    x_haken = 260
    x_max = 300

    y_min = -80
    y_haken = -40
    y_max = -20

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_haken, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Kastenstartposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kastenposition 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kastenposition 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kastenposition 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kastenposition 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kastenposition 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Hochfahren')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Hakenstartposition')
    bot.move_to(x_haken, y_max, z_max, 0)

    print('Position 1')
    bot.move_to(x_haken, y_max, z_min, 0)

    print('Position 2')
    bot.slide_to(x_min, y_haken, z_min, 0)

    print('Position 3')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Zurück nach Position 1')
    bot.move_to(x_max, y_min, z_max, 0)

def hammer(bot):

    print("Aktion: Hammer")
    x_min = 220
    x_max = 300

    y_min = 20
    y_max = 80

    z_min = -72.5
    z_max = -8

    print('Hochfahren')
    bot.move_to(x_max, 0, z_max, 0)

    print('Homing-Modus')
    bot.home()

    print('Startposition')
    bot.move_to(x_max, y_min, z_max, 0)

    print('Kasten-Position 1')
    bot.move_to(x_max, y_min, z_min, 0)

    print('Kasten-Position 2')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Kasten-Position 3')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Kasten-Position 4')
    bot.slide_to(x_max, y_max, z_min, 0)

    print('Kasten-Position 1')
    bot.slide_to(x_max, y_min, z_min, 0)

    print('Kreuz-Position 1')
    bot.slide_to(x_min, y_max, z_min, 0)

    print('Hochfahren 2')
    bot.move_to(x_min, y_max, z_max, 0)

    print('Startposition 2')
    bot.move_to(x_max, y_max, z_max, 0)

    print('Kreuz-Position 2')
    bot.move_to(x_max, y_max, z_min, 0)

    print('Kreuz-Position 3')
    bot.slide_to(x_min, y_min, z_min, 0)

    print('Hochfahren 3')
    bot.move_to(x_min, y_min, z_max, 0)

if overall_pred == "Wischen":
    wischen(bot)
else:
    hammer(bot)